# Scrape Wallstreet online

In [1]:
from ModulScrapeWallstreet import scrape_futureprice



In [2]:
future_df = scrape_futureprice()

C:\Users\Lenovo\Documents\GitHub\ContractCalculator\ModulScrapeWallstreet.py:15: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 15 of the file C:\Users\Lenovo\Documents\GitHub\ContractCalculator\ModulScrapeWallstreet.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(page.content, 'html')
C:\Users\Lenovo\Documents\GitHub\ContractCalculator\ModulScrapeWallstreet.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  future_price_today = future_price_today.append(ser, ignore_index=True)
C:\Users\Lenovo\Documents\GitHub\Contract

In [3]:
future_df

,kontrakt,price,absolut_inc,perc_inc,date_fullfillment,date_price,currency,commodity_id
0,Mai 2022,361.00,"9,20","+2,62 %",2022-05-23,2022-05-16,GBP,2
1,Jul 2022,360.05,"12,45","+3,58 %",2022-07-07,2022-05-16,GBP,2
2,Nov 2022,352.00,"15,70","+4,67 %",2022-11-23,2022-05-16,GBP,2
3,Jan 2023,354.00,"15,30","+4,52 %",2023-01-23,2022-05-16,GBP,2
4,Mär 2023,355.30,"14,90","+4,38 %",2023-03-23,2022-05-16,GBP,2
5,Mai 2023,356.80,"14,50","+4,24 %",2023-05-23,2022-05-16,GBP,2
6,Jul 2023,327.10,"10,25","+3,23 %",2023-07-07,2022-05-16,GBP,2
7,Nov 2023,292.80,"6,00","+2,09 %",2023-11-23,2022-05-16,GBP,2
8,Jan 2024,293.75,"6,00","+2,09 %",2024-01-23,2022-05-16,GBP,2
9,Mär 2024,295.25,"6,00","+2,07 %",2024-03-22,2022-05-16,GBP,2


In [4]:
future_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   kontrakt           10 non-null     object        
 1   price              10 non-null     float64       
 2   absolut_inc        10 non-null     object        
 3   perc_inc           10 non-null     object        
 4   date_fullfillment  10 non-null     datetime64[ns]
 5   date_price         10 non-null     datetime64[ns]
 6   currency           10 non-null     object        
 7   commodity_id       10 non-null     int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 768.0+ bytes


In [5]:
tosql_df = future_df[['commodity_id', 'date_fullfillment', 'date_price', 'price', 'currency']]

In [6]:
import sqlite3
def future_price_to_sql(future_price_today):
    con = sqlite3.connect('contrcalc.db') 
    future_price_today.to_sql('price_table', con, if_exists='append', index=False)
    return
future_price_to_sql(tosql_df)

In [7]:
future_df.to_csv('Data/future_df.csv')

In [8]:
import pandas as pd
con = sqlite3.connect('contrcalc.db') 
price_df = pd.read_sql('Select * FROM price_table', con, index_col='price_id')

In [10]:
price_df

,commodity_id,date_fullfillment,date_price,price,currency
price_id,,,,,
1,2,2022-05-23 00:00:00,2022-05-16 00:00:00,361.00,GBP
2,2,2022-07-07 00:00:00,2022-05-16 00:00:00,360.05,GBP
3,2,2022-11-23 00:00:00,2022-05-16 00:00:00,352.00,GBP
4,2,2023-01-23 00:00:00,2022-05-16 00:00:00,354.00,GBP
5,2,2023-03-23 00:00:00,2022-05-16 00:00:00,355.30,GBP
6,2,2023-05-23 00:00:00,2022-05-16 00:00:00,356.80,GBP
7,2,2023-07-07 00:00:00,2022-05-16 00:00:00,327.10,GBP
8,2,2023-11-23 00:00:00,2022-05-16 00:00:00,292.80,GBP
9,2,2024-01-23 00:00:00,2022-05-16 00:00:00,293.75,GBP
